In [ ]:
import json, requests, time
from sqlalchemy import create_engine
import pandas as pd

engine = create_engine('postgresql://postgres:argmax@pg:5432/postgres')


# Data
Every time a user opens a mobile app, an auction is going on behind the scenes. The highest bidder gets to advertise his ad to the user.
## Auctions Table

In [ ]:
sql_query = 'SELECT * FROM auctions;'
with engine.connect() as db_con:
    big_df = pd.read_sql(sql_query, con=db_con)

big_df

## App Vectors table
We've gathered the first few sentences from the app store description and embedded it with a [model](https://huggingface.co/mixedbread-ai/mxbai-embed-large-v1)

In [ ]:
sql_query = f'''
SELECT
    *
FROM app_vectors
'''
has_embedding = False
while not has_embedding:
    with engine.connect() as db_con:
        embeds_df = pd.read_sql(sql_query, con=db_con)
    has_embedding = (~df["embedding"].isna()).all()
    if not has_embedding:
        print("Waiting for embeddings...")
        time.sleep(15)

embeds_df


We can use the `<=>` operator to run vector search within the database

In [ ]:

vec = json.loads(df.embedding[0]) # get the first embedding
print ("Embedding size: {l}".format(l=len(vec)))

sql_query = f'''
SELECT
    "bundleId"
FROM app_vectors
ORDER BY embedding<=>'{json.dumps(vec)}'
'''
with engine.connect() as db_con:
    df = pd.read_sql(sql_query, con=db_con)

df

# What you need to do
## The hypothesis
We assume that apps with similar desciptions, would have a similar asking price in the auctions (`sentPrice` column).

Use cosine similarity (`<=>`) on the embeddings to find similar apps, and any statistical tools you find suitable to prove or disprove this hypothesis.

## Is it consistent?
There are several other features in the auctions table (such as `CountryCode` and `OS`), 
Do your findings hold for those as well?

In [ ]:
# TODO: Please write your analysis here

# Solution <br>
### 1. We need to find a way to quantify how good this way of searching is. <br>
Lets calculate the correlation coefficient between cosine scores and price differences, then we can check if the correlation is statistically significant.

### 1.1 Correlation between the cosine similarity score of an app with its most similar app, and the price difference

In [ ]:
import numpy as np
from scipy.stats import pearsonr

def statistical_tests_v1():
    ids = embeds_df['id'].to_list()
    most_similar_app_id = get_similar_app_id(ids) # {app_id: most_similar_app_id}
    most_similar_app_score = get_similar_app_score(ids) # {app_id: most_similar_app_cosine_score}
    price_difference = get_price_difference(most_similar_app_id) # {app_id: price_difference_with_most_similar_app}
    assert list(most_similar_app_id.keys()) == list(price_difference.keys()), 'ids are in a different order'
    
    # Pearson
    corr_coeff, p_value = pearsonr(list(most_similar_app_score.values()), list(price_difference.values()))
    print("Pearson correlation coefficient:", corr_coeff)
    print("P-value:", p_value)
    alpha = 0.05
    if p_value < alpha:
        print("The correlation is statistically significant.\n")
    else:
        print("The correlation is not statistically significant.\n")

    return list(most_similar_app_score.values()), list(price_difference.values())

def get_similar_app_id(ids):
    most_similar_app_id = {_id: get_most_similar(_id, ids)[0] for _id in ids}
    return most_similar_app_id

def get_similar_app_score(ids):
    most_similar_app_score = {_id: get_most_similar(_id, ids)[1] for _id in ids}
    return most_similar_app_score

def get_most_similar(_id, ids):
    app_embeds = embeds_df[embeds_df['id'] == _id]['embedding'].item()
    other_apps_embeds = {other_app_id: embeds_df[embeds_df['id'] == other_app_id]['embedding'].item() for other_app_id in ids if other_app_id != _id}
    max_score = np.NINF
    most_similar_id = None
    for other_app_id, other_app_embeds in other_apps_embeds.items():
        cosine_score = cosine_similarity_score(app_embeds, other_app_embeds)
        if cosine_score > max_score:
            max_score = cosine_score
            most_similar_id = other_app_id
    return (most_similar_id, max_score)

def cosine_similarity_score(vector1, vector2):
    vector1 = json.loads(vector1)
    vector2 = json.loads(vector2)
    
    dot_product = np.dot(vector1, vector2)
    norm_vector1 = np.linalg.norm(vector1)
    norm_vector2 = np.linalg.norm(vector2)
    similarity = dot_product / (norm_vector1 * norm_vector2)
    return similarity

def get_price_difference(most_similar_app_id):
    price_difference = {}
    for app_id, other_app_id in most_similar_app_id.items():
        price_difference[app_id] = abs(big_df[big_df['id'] == app_id]['sentPrice'].item() - big_df[big_df['id'] == other_app_id]['sentPrice'].item())
    return price_difference

In [ ]:
sims, diffs = statistical_tests_v1()

In [ ]:
# !pip install scipy
# !pip install sentence_transformers

### Discussion

As there is'nt a lot of data (only 18 records in the app_vectors table), we can't really tell something here. We would expect that as the similarity gets higher, the differences in price will get smaller. As it can be seen this is not the case here, probably because of the small sample size. That's why we can try a different approach next, where we check not only for an app with it's most similar app, but with every other app and then calculate the correlation (we would have 18 * 17 = 306 records which is slightly better).

### 1.2 Correlation between the cosine similarity score of an app with every other app, and the price difference

In [ ]:
import numpy as np
from scipy.stats import pearsonr

def statistical_tests_v2():
    scores = {}
    price_difference = {}
    ids = embeds_df['id'].to_list()
    for id1 in ids:
        vec1 = embeds_df[embeds_df['id'] == id1]['embedding'].item()
        for id2 in ids:
            if id1 == id2:
                continue
            vec2 = embeds_df[embeds_df['id'] == id2]['embedding'].item()
            scores[(id1, id2)] = cosine_similarity_score(vec1, vec2)
            price_difference[(id1, id2)] = abs(big_df[big_df['id'] == id1]['sentPrice'].item() - big_df[big_df['id'] == id2]['sentPrice'].item())
    assert list(scores.keys()) == list(price_difference.keys()), 'ids are in a different order'

    # Pearson
    corr_coeff, p_value = pearsonr(list(scores.values()), list(price_difference.values()))
    print("Pearson correlation coefficient:", corr_coeff)
    print("P-value:", p_value)
    alpha = 0.05
    if p_value < alpha:
        print("The correlation is statistically significant.\n")
    else:
        print("The correlation is not statistically significant.\n")

    return (list(scores.values()), list(price_difference.values()))

In [ ]:
scores, diffs = statistical_tests_v2()

In [ ]:
import matplotlib.pyplot as plt

plt.scatter(scores, diffs)

plt.xlabel('scores')
plt.ylabel('price diffs')

z = np.polyfit(scores, diffs, 1)
p = np.poly1d(z)
plt.plot(scores,p(scores),"r--")

### Discussion

At a 95% confidence level, we fail to reject the null hypothesis, suggesting that the correlation observed is not statistically significant (we can see that it's negative as we would expect but it's small). However, at a 90% confidence level, we can reject the null hypothesis. Whether this finding is beneficial depends on our specific objectives and the error rate we can afford.

### 2. Is it consistent?

Intuitively, other features such as OS and CountryCode might provide more insightful comparisons between apps. For instance, applications with similar functionalities, alongside metrics like 'number of downloads,' could get us more meaningful insights. A similar number of downloads typically signifies a successful app, hence influencing a potentially higher sentPrice. To check this assumption, we can train a linear regression model. By inputting two apps with comparable features, we would expect observing a closer similarity in sentPrice values, thus affirming our hypothesis.